In [1]:
import numpy as np
import sympy as sp

In [35]:
# fixed Params
K1 = 45  # thermal conductivity of the pans(steel)

d = 0.444 # diameter of pans

t = 0.005  # thickness of the pan

h = 87 #convicitve coefficient of flue gases 

cp = 1.4 * (10**3) #specific heat at constant pressure

mf = 6.74 #mass flow rate of flue gases

In [36]:
L = 4
P = round(np.pi * d, 3)

For conduction we are taking conduction through pans

In [37]:
A = round(np.pi * (d ** 2) / 4, 3)
kxx = (A * K1 )/ t

In [38]:
kxx

1395.0

In [39]:
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt
k = kxx * Matrix([[1,-1], [-1,1]])

In [40]:
k

Matrix([
[ 1395.0, -1395.0],
[-1395.0,  1395.0]])

For conviective heat transfer 
- we are consedering end convection through pan
- parametric convection through chimney

In [41]:
conv_mat_pan = (h * A )* Matrix([[1,0],[0,0]])

In [42]:
conv_mat = (h * P * L)/6 * Matrix([[2,1],[1,2]])

In [43]:
conv_mat

Matrix([
[161.82,  80.91],
[ 80.91, 161.82]])

In [44]:
conv_mat_pan

Matrix([
[13.485, 0],
[     0, 0]])

Calculating conduction and convection matrices for both pan and chimney
- k1, k2, k3 are for pan 1, pan 2, pan 3 respectively
- k4 is for chimney

In [45]:
k1 = k2 = k3 = (k + conv_mat_pan)
k1

Matrix([
[1408.485, -1395.0],
[ -1395.0,  1395.0]])

In [46]:
k4 = k + conv_mat

In [47]:
k4

Matrix([
[ 1556.82, -1314.09],
[-1314.09,  1556.82]])

Now calculating Global heat transfet matrix `K`

In [48]:
K = Matrix([
    [k1[0,0], k1[0,1],0,0,0],
    [k1[1,0], k1[1,1] + k2[0,0],k2[0,1],0,0],
    [0,k2[1,0],k2[1,1] + k3[0,0], k3[0,1],0],
     [0,0,k3[1,0],k3[1,1]+k4[0,0],k4[0,1]],
     [0,0,0,k4[1,0], k4[1,1]]])

In [49]:
K

Matrix([
[1408.485,  -1395.0,        0,        0,        0],
[ -1395.0, 2803.485,  -1395.0,        0,        0],
[       0,  -1395.0, 2803.485,  -1395.0,        0],
[       0,        0,  -1395.0,  2951.82, -1314.09],
[       0,        0,        0, -1314.09,  1556.82]])

Now creating matrices for the nodal temperatures and heat flow rates

In [50]:
T2,T3,T4,T5 = sp.symbols("T2 T3 T4 T5")

In [51]:
temp = Matrix([[1600],[T2],[T3],[T4], [573]])
temp

Matrix([
[1600],
[  T2],
[  T3],
[  T4],
[ 573]])

In [52]:
Q1,Q2,Q3,Q4,Q5 = sp.symbols("Q1 Q2 Q3 Q4 Q5")

In [53]:
Q = Matrix([[Q1],[Q2],[Q3],[Q4], [Q5]])
Q

Matrix([
[Q1],
[Q2],
[Q3],
[Q4],
[Q5]])

Calculating global temperature distribution along each node

In [54]:
f = K * temp
f

Matrix([
[               2253576.0 - 1395.0*T2],
[ 2803.485*T2 - 1395.0*T3 - 2232000.0],
[-1395.0*T2 + 2803.485*T3 - 1395.0*T4],
[ -1395.0*T3 + 2951.82*T4 - 752973.57],
[              892057.86 - 1314.09*T4]])

In [55]:
q = Q.subs([[Q2,0],[Q3,0],[Q4,0]])

In [56]:
eqn = sp.solve((f[2], f[3],f[4]))

In [57]:
eqn

{T2: 1123.14907387261, T3: 896.661057324841, T4: 678.840764331210}

In [58]:
temp = np.array(list(eqn.values()))
temp

array([1123.14907387261, 896.661057324841, 678.840764331210], dtype=object)

Creating anonymous function to find heatflow rate through different nodes

In [59]:
heatFlowRate = lambda temp,cp, mf:  mf * cp * (temp - 300)

In [64]:
heatFlow = [heatFlowRate(temp[0],cp,mf), heatFlowRate(temp[1],cp,mf), heatFlowRate(temp[2],cp,mf), heatFlowRate(573, cp,mf)]

In [65]:
heatFlow

[7767234.66106196, 5630093.73691720, 3574741.45222930, 2576028.0]

In [66]:
efficiency = lambda heats: sum(heats)/(mf*(19605*(10**3)))

In [67]:
round(efficiency(heatFlow),3)

0.148